# Teil 3: Erweiterte Remote-Ausführungstools

Im letzten Abschnitt haben wir ein Spielzeugmodell mit Federated Learning trainiert. Dazu haben wir .send () und .get () für unser Modell aufgerufen, an den Speicherort der Trainingsdaten gesendet, aktualisiert und dann zurückgebracht. Am Ende des Beispiels wurde uns jedoch klar, dass wir etwas weiter gehen müssen, um die Privatsphäre der Menschen zu schützen. Wir wollen nämlich die Verläufe ** vor dem Aufruf von .get () mitteln. Auf diese Weise werden wir niemals den genauen Gradienten von jemandem sehen (wodurch die Privatsphäre besser geschützt wird !!!)

Dafür brauchen wir aber noch ein paar Stücke:

- Verwenden Sie einen Zeiger, um einen Tensor direkt an einen anderen Mitarbeiter zu senden

Während wir hier sind, werden wir außerdem einige fortgeschrittenere Tensoroperationen kennenlernen, die uns sowohl bei diesem Beispiel als auch bei einigen in Zukunft helfen werden!

Autoren:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)

Übersetzer:
- Vineet Jain - Github: [@vineetjai](https://github.com/vineetjai)

In [1]:
import torch
import syft as sy
hook = sy.TorchHook(torch)

# Abschnitt 3.1 - Zeiger auf Zeiger

Wie Sie wissen, fühlen sich PointerTensor-Objekte wie normale Tensoren an. Tatsächlich sind sie so ähnlich wie Tensoren, dass wir sogar Zeiger ** auf ** die Zeiger haben können. Hör zu!

In [2]:
bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')

In [3]:
# this is a local tensor
x = torch.tensor([1,2,3,4])
x

tensor([1, 2, 3, 4])

In [4]:
# this sends the local tensor to Bob
x_ptr = x.send(bob)

# this is now a pointer
x_ptr

(Wrapper)>[PointerTensor | me:31389458232 -> bob:63481378785]

In [5]:
# now we can SEND THE POINTER to alice!!!
pointer_to_x_ptr = x_ptr.send(alice)

pointer_to_x_ptr

(Wrapper)>[PointerTensor | me:72793192181 -> alice:31389458232]

### Was ist passiert?

Im vorherigen Beispiel haben wir einen Tensor namens "x" erstellt und an Bob gesendet, wobei ein Zeiger auf unserem lokalen Computer (`x_ptr`) erstellt wurde.

Dann haben wir `x_ptr.send (alice)` aufgerufen, was ** den Zeiger ** an Alice gesendet hat.

Beachten Sie, dass dies die Daten NICHT verschoben hat! Stattdessen hat es den Zeiger auf die Daten bewegt !!

In [6]:
# As you can see above, Bob still has the actual data (data is always stored in a LocalTensor type). 
bob._objects

{63481378785: tensor([1, 2, 3, 4])}

In [7]:
# Alice, on the other hand, has x_ptr!! (notice how it points at bob)
alice._objects

{31389458232: (Wrapper)>[PointerTensor | alice:31389458232 -> bob:63481378785]}

In [8]:
# and we can use .get() to get x_ptr back from Alice

x_ptr = pointer_to_x_ptr.get()
x_ptr

(Wrapper)>[PointerTensor | me:31389458232 -> bob:63481378785]

In [9]:
# and then we can use x_ptr to get x back from Bob!

x = x_ptr.get()
x

tensor([1, 2, 3, 4])

### Arithmetik auf Zeiger -> Zeiger -> Datenobjekt

Und genau wie bei normalen Zeigern können wir beliebige PyTorch Operationen über diese Tensoren ausführen

In [10]:
bob._objects

{}

In [11]:
alice._objects

{}

In [12]:
p2p2x = torch.tensor([1,2,3,4,5]).send(bob).send(alice)

y = p2p2x + p2p2x

In [13]:
bob._objects

{18083425258: tensor([1, 2, 3, 4, 5]),
 61451767724: tensor([ 2,  4,  6,  8, 10])}

In [14]:
alice._objects

{84068841042: (Wrapper)>[PointerTensor | alice:84068841042 -> bob:18083425258],
 57717534270: (Wrapper)>[PointerTensor | alice:57717534270 -> bob:61451767724]}

In [15]:
y.get().get()

tensor([ 2,  4,  6,  8, 10])

In [16]:
bob._objects

{18083425258: tensor([1, 2, 3, 4, 5])}

In [17]:
alice._objects

{84068841042: (Wrapper)>[PointerTensor | alice:84068841042 -> bob:18083425258]}

In [18]:
p2p2x.get().get()

tensor([1, 2, 3, 4, 5])

In [19]:
bob._objects

{}

In [20]:
alice._objects

{}

# Abschnitt 3.2 - Zeigerkettenoperationen

Wenn wir also im letzten Abschnitt eine .send() - oder eine .get() -Operation aufriefen, wurde diese Operation direkt auf dem Tensor auf unserer lokalen Maschine aufgerufen. Wenn Sie jedoch eine Zeigerkette haben, möchten Sie manchmal Operationen wie .get() oder .send() für den ** letzten ** Zeiger in der Kette aufrufen (z. B. das direkte Senden von Daten von einem Worker an einen anderen). Um dies zu erreichen, möchten Sie Funktionen verwenden, die speziell für diesen Vorgang zum Schutz der Privatsphäre entwickelt wurden.

Diese Operationen sind:

- `my_pointer2pointer.move(another_worker)`

In [21]:
# x is now a pointer to the data which lives on Bob's machine
x = torch.tensor([1,2,3,4,5]).send(bob)

In [22]:
print('  bob:', bob._objects)
print('alice:',alice._objects)

  bob: {3913539550: tensor([1, 2, 3, 4, 5])}
alice: {}


In [23]:
x = x.move(alice)

In [24]:
print('  bob:', bob._objects)
print('alice:',alice._objects)

  bob: {}
alice: {33856269530: tensor([1, 2, 3, 4, 5])}


In [25]:
x

(Wrapper)>[PointerTensor | me:33856269530 -> alice:33856269530]

Ausgezeichnet! Jetzt sind wir mit den Tools ausgestattet, mit denen Sie mithilfe eines vertrauenswürdigen Aggregators eine Remote-Gradientenmittelung durchführen können!

# Herzliche Glückwünsche!!! - Zeit, der Community beizutreten!

Herzlichen Glückwunsch zum Abschluss dieses Notizbuch-Tutorials! Wenn Ihnen dies gefallen hat und Sie sich der Bewegung zur Wahrung der Privatsphäre, zum dezentralen Besitz von KI und der KI-Lieferkette (Daten) anschließen möchten, können Sie dies auf folgende Weise tun!

### Star PySyft auf GitHub

Der einfachste Weg, unserer Community zu helfen, besteht darin, die Repos in der Hauptrolle zu spielen! Dies hilft, das Bewusstsein für die coolen Tools zu schärfen, die wir bauen.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Mach mit bei unserem Slack!

Der beste Weg, um über die neuesten Entwicklungen auf dem Laufenden zu bleiben, ist, sich unserer Community anzuschließen! Sie können dies tun, indem Sie das Formular unter [http://slack.openmined.org](http://slack.openmined.org) ausfüllen.

### Treten Sie einem Code-Projekt bei!

Der beste Weg, um zu unserer Community beizutragen, besteht darin, Code-Mitwirkender zu werden! Sie können jederzeit zur Seite PySyft GitHub Issues gehen und nach "Projekten" filtern. Dies zeigt Ihnen alle Top-Level-Tickets und gibt einen Überblick darüber, an welchen Projekten Sie teilnehmen können! Wenn Sie nicht an einem Projekt teilnehmen möchten, aber ein wenig programmieren möchten, können Sie auch nach weiteren "einmaligen" Miniprojekten suchen, indem Sie nach GitHub-Problemen suchen, die als "gute erste Ausgabe" gekennzeichnet sind.

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)



### Spenden

Wenn Sie keine Zeit haben, zu unserer Codebasis beizutragen, aber dennoch Unterstützung leisten möchten, können Sie auch Unterstützer unseres Open Collective werden. Alle Spenden fließen in unser Webhosting und andere Community-Ausgaben wie Hackathons und Meetups!

[Open Collective Open Collective Page](https://opencollective.com/openmined)